**Part 4 – Mini System Design**

Design a lightweight Sales Insight & Alert System.

Include:

High-level architecture (diagram optional)

Data flow

Example alerts or insights


How often it runs

Failure cases and limitations

Think like:
“If SkyGeni were to productize this, what would it look like?”


**High-level architecture (diagram optional)**

1. data preprocessing & feature layer
2. risk scoring engine
3. insight & alert layer
4. sales dashboard / notification

my mini systm design is when sales data flows in , its preprocessed and risk signals are computed . Insights and alerts are generated . Sales leaders take action .

1. **data Preprocessing**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def preprocess_sales_data(df):
    df = df.copy()

    df['created_date'] = pd.to_datetime(df['created_date'], dayfirst=True)
    df['closed_date'] = pd.to_datetime(df['closed_date'], dayfirst=True)

    df = df[df['sales_cycle_days'] >= 0]
    df['is_won'] = df['outcome'].apply(lambda x: 1 if x == 'Won' else 0)

    return df


the above function preprocesses the sales data by converting date columns to datetime format, filtering out any deals with negative sales cycle days, and creating a binary 'is_won' column based on the 'outcome'.

2. **Risk Scoring Engine**

In [3]:
def build_deal_risk_engine(df):
    df = df.copy()

    avg_cycle = df['sales_cycle_days'].mean()

    lead_win_rate = df.groupby('lead_source')['is_won'].mean()
    low_win_sources = lead_win_rate[lead_win_rate < lead_win_rate.mean()].index

    region_win_rate = df.groupby('region')['is_won'].mean()
    low_win_regions = region_win_rate[region_win_rate < region_win_rate.mean()].index

    df['risk_long_cycle'] = df['sales_cycle_days'] > avg_cycle
    df['risk_lead_source'] = df['lead_source'].isin(low_win_sources)
    df['risk_region'] = df['region'].isin(low_win_regions)

    df['risk_score'] = (
        df['risk_long_cycle'].astype(int) +
        df['risk_lead_source'].astype(int) +
        df['risk_region'].astype(int)
    )

    def map_risk(score):
        if score == 0:
            return 'Low Risk'
        elif score in [1, 2]:
            return 'Medium Risk'
        else:
            return 'High Risk'

    df['risk_level'] = df['risk_score'].apply(map_risk)

    return df


the above fucntion build_deal_risk_engine is a simple rule-based risk engine that identifies deals at risk of being lost based on sales cycle length, lead source, and region. 
It calculates a risk score and categorizes deals into low, medium, or high risk levels.

3. **Insight Generation**

A . Top 10 High-Risk Open Deals

In [4]:
def get_top_risky_open_deals(df, top_n=10):
    open_deals = df[df['outcome'] != 'Won']

    return (
        open_deals[open_deals['risk_level'] == 'High Risk']
        .sort_values(['risk_score', 'sales_cycle_days'], ascending=False)
        .head(top_n)
        [['deal_id', 'sales_rep_id', 'region', 'lead_source',
          'deal_stage', 'sales_cycle_days', 'deal_amount', 'risk_level']]
    )


the above funciton gets the top 10 risky open deals based on the risk engine we built. 
It filters for open deals, then selects those marked as 'High Risk', sorts them by risk score and sales cycle days, and returns the relevant details for those deals.

B. Low performing segments

In [5]:
def get_low_performing_segments(df):
    return (
        df.groupby(['region', 'lead_source'])['is_won'].mean()
        .reset_index(name='win_rate').sort_values('win_rate').head(10))


the above fucntion get low performing segments by region and lead source. 
It groups the data by region and lead source, calculates the win rate for each segment, and then returns the top 10 segments with the lowest win rates.

C . Rep  level coaching signals

In [6]:
def get_rep_coaching_signals(df):
    rep_stats = (
        df.groupby('sales_rep_id').agg(
            win_rate=('is_won', 'mean'),
            avg_sales_cycle=('sales_cycle_days', 'mean'),
            total_deals=('deal_id', 'count')
        ).reset_index())

    avg_win_rate = df['is_won'].mean()
    avg_cycle = df['sales_cycle_days'].mean()

    return rep_stats[
        (rep_stats['win_rate'] < avg_win_rate) &
        (rep_stats['avg_sales_cycle'] > avg_cycle)
    ].sort_values('win_rate')


th e above fucntion get rep coaching signals identifies sales representatives who may benefit from additional coaching by analyzing their win rates and average sales cycle durations compared to overall averages.

In [7]:
df= pd.read_csv("skygeni_sales_data.csv")

In [8]:
df = preprocess_sales_data(df)
df = build_deal_risk_engine(df)

top_risky_deals = get_top_risky_open_deals(df)
low_segments = get_low_performing_segments(df)
rep_coaching = get_rep_coaching_signals(df)

C:\Users\anjan\AppData\Local\Temp\ipykernel_2556\1802838225.py:4: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['created_date'] = pd.to_datetime(df['created_date'], dayfirst=True)
C:\Users\anjan\AppData\Local\Temp\ipykernel_2556\1802838225.py:5: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['closed_date'] = pd.to_datetime(df['closed_date'], dayfirst=True)


In [9]:
top_risky_deals

,deal_id,sales_rep_id,region,lead_source,deal_stage,sales_cycle_days,deal_amount,risk_level
3456,D03457,rep_24,North America,Partner,Qualified,120,4130,High Risk
4347,D04348,rep_5,APAC,Partner,Negotiation,120,12027,High Risk
4878,D04879,rep_5,APAC,Partner,Demo,120,96732,High Risk
4987,D04988,rep_5,North America,Partner,Demo,120,8238,High Risk
931,D00932,rep_2,North America,Partner,Negotiation,119,4518,High Risk
1948,D01949,rep_15,North America,Partner,Demo,119,6208,High Risk
4140,D04141,rep_5,APAC,Partner,Qualified,119,3397,High Risk
1209,D01210,rep_21,APAC,Partner,Qualified,118,19494,High Risk
2064,D02065,rep_9,North America,Partner,Closed,118,21629,High Risk
2724,D02725,rep_12,North America,Partner,Proposal,118,9654,High Risk


In [10]:
low_segments

,region,lead_source,win_rate
14,North America,Partner,0.390625
1,APAC,Outbound,0.414791
7,Europe,Referral,0.429032
8,India,Inbound,0.429825
13,North America,Outbound,0.435374
3,APAC,Referral,0.435737
4,Europe,Inbound,0.437500
6,Europe,Partner,0.442568
10,India,Partner,0.455128
9,India,Outbound,0.455975


In [11]:
rep_coaching

,sales_rep_id,win_rate,avg_sales_cycle,total_deals
1,rep_10,0.424084,63.853403,191
21,rep_6,0.427861,64.781095,201
4,rep_13,0.441026,68.456410,195
6,rep_15,0.445714,66.817143,175
24,rep_9,0.446237,63.822581,186


4 . **Alert**

A. deal alert

In [12]:
def generate_deal_alerts(df):
    alerts = []

    for _, row in df[df['risk_level'] == 'High Risk'].iterrows():
        alerts.append(
            f"🚨 Deal {row['deal_id']} is HIGH RISK "
            f"({row['sales_cycle_days']} days, "
            f"{row['lead_source']} lead, {row['region']})")

    return alerts

B. Rep alerts 

In [13]:
def generate_rep_alerts(rep_df):
    return [
        f"📉 Rep {r.sales_rep_id} needs coaching | "
        f"Win rate: {r.win_rate:.2f}, "
        f"Avg cycle: {r.avg_sales_cycle:.1f} days"
        for _, r in rep_df.iterrows()]

C . **Weekly Executive Summary**

In [14]:
def weekly_summary(df):
    return {
        "total_deals": len(df),
        "overall_win_rate": round(df['is_won'].mean(), 2),
        "high_risk_deals": int((df['risk_level'] == 'High Risk').sum()),
        "avg_sales_cycle_days": round(df['sales_cycle_days'].mean(), 1)}


In [15]:
df = preprocess_sales_data(df)
df = build_deal_risk_engine(df)

top_risky_deals = get_top_risky_open_deals(df)
low_segments = get_low_performing_segments(df)
rep_coaching = get_rep_coaching_signals(df)

deal_alerts = generate_deal_alerts(df)
rep_alerts = generate_rep_alerts(rep_coaching)

weekly_report = weekly_summary(df)
print("Weekly Sales Performance Summary:")
for k, v in weekly_report.items():
    print(f"{k.replace('_', ' ').title()}: {v}")

Weekly Sales Performance Summary:
Total Deals: 5000
Overall Win Rate: 0.45
High Risk Deals: 304
Avg Sales Cycle Days: 63.8


In [20]:
def weekly_sales_insight_job(df):
    df = preprocess_sales_data(df)
    df = build_deal_risk_engine(df)

    top_risky_deals = get_top_risky_open_deals(df)
    low_segments = get_low_performing_segments(df)
    rep_coaching = get_rep_coaching_signals(df)

    deal_alerts = generate_deal_alerts(df)
    rep_alerts = generate_rep_alerts(rep_coaching)

    weekly_report = weekly_summary(df)

    print("Weekly Sales Performance Summary:")
    for k, v in weekly_report.items():
        print(f"{k.replace('_', ' ').title()}: {v}")

    return {
        "summary": weekly_report,
        "top_risky_deals": top_risky_deals,
        "low_segments": low_segments,
        "rep_alerts": rep_alerts,
        "deal_alerts": deal_alerts }


this is the end function , the overall risk scoring engine so while execution kindly read the dataset into a variable df and use the below function . 

i have executed the function with the dataset provided  , ahve read the same above

In [21]:
result = weekly_sales_insight_job(df)

print("\n--- Weekly Summary ---")
for k, v in result["summary"].items():
    print(f"{k.replace('_', ' ').title()}: {v}")

print("\n--- Top Risky Deals ---")
display(result["top_risky_deals"])

print("\n--- Low Performing Segments ---")
display(result["low_segments"])

print("\n--- Rep Alerts ---")
for alert in result["rep_alerts"]:
    print(alert)

print("\n--- Deal Alerts ---")
for alert in result["deal_alerts"]:
    print(alert)


Weekly Sales Performance Summary:
Total Deals: 5000
Overall Win Rate: 0.45
High Risk Deals: 304
Avg Sales Cycle Days: 63.8

--- Weekly Summary ---
Total Deals: 5000
Overall Win Rate: 0.45
High Risk Deals: 304
Avg Sales Cycle Days: 63.8

--- Top Risky Deals ---


,deal_id,sales_rep_id,region,lead_source,deal_stage,sales_cycle_days,deal_amount,risk_level
3456,D03457,rep_24,North America,Partner,Qualified,120,4130,High Risk
4347,D04348,rep_5,APAC,Partner,Negotiation,120,12027,High Risk
4878,D04879,rep_5,APAC,Partner,Demo,120,96732,High Risk
4987,D04988,rep_5,North America,Partner,Demo,120,8238,High Risk
931,D00932,rep_2,North America,Partner,Negotiation,119,4518,High Risk
1948,D01949,rep_15,North America,Partner,Demo,119,6208,High Risk
4140,D04141,rep_5,APAC,Partner,Qualified,119,3397,High Risk
1209,D01210,rep_21,APAC,Partner,Qualified,118,19494,High Risk
2064,D02065,rep_9,North America,Partner,Closed,118,21629,High Risk
2724,D02725,rep_12,North America,Partner,Proposal,118,9654,High Risk



--- Low Performing Segments ---


,region,lead_source,win_rate
14,North America,Partner,0.390625
1,APAC,Outbound,0.414791
7,Europe,Referral,0.429032
8,India,Inbound,0.429825
13,North America,Outbound,0.435374
3,APAC,Referral,0.435737
4,Europe,Inbound,0.437500
6,Europe,Partner,0.442568
10,India,Partner,0.455128
9,India,Outbound,0.455975



--- Rep Alerts ---
📉 Rep rep_10 needs coaching | Win rate: 0.42, Avg cycle: 63.9 days
📉 Rep rep_6 needs coaching | Win rate: 0.43, Avg cycle: 64.8 days
📉 Rep rep_13 needs coaching | Win rate: 0.44, Avg cycle: 68.5 days
📉 Rep rep_15 needs coaching | Win rate: 0.45, Avg cycle: 66.8 days
📉 Rep rep_9 needs coaching | Win rate: 0.45, Avg cycle: 63.8 days

--- Deal Alerts ---
🚨 Deal D00054 is HIGH RISK (92 days, Partner lead, North America)
🚨 Deal D00056 is HIGH RISK (94 days, Partner lead, North America)
🚨 Deal D00072 is HIGH RISK (112 days, Partner lead, North America)
🚨 Deal D00075 is HIGH RISK (90 days, Partner lead, APAC)
🚨 Deal D00087 is HIGH RISK (102 days, Partner lead, North America)
🚨 Deal D00096 is HIGH RISK (93 days, Partner lead, North America)
🚨 Deal D00107 is HIGH RISK (94 days, Partner lead, North America)
🚨 Deal D00145 is HIGH RISK (107 days, Partner lead, APAC)
🚨 Deal D00158 is HIGH RISK (70 days, Partner lead, North America)
🚨 Deal D00181 is HIGH RISK (93 days, Partner le

Weekly scheduled run 

In [17]:
# import schedule
# import time
# schedule.every().monday.at("09:00").do(weekly_sales_insight_job, df=df)

# while True:
#     schedule.run_pending()
#     time.sleep(60)


**----------------------------------------END----------------------------------------------**

**Conclusion :**

I chose a rule-based approach because the goal here is decision support, not pure prediction accuracy. 

Sales leaders need to understand why a deal is risky, not just see a probability number.

Rule-based logic makes every decision transparent and easy to explain, which builds trust and makes the insights easier to act on.


**----------------------------------------EXTRA----------------------------------------------**

below i have added the functions for new incoming data which doesnt have an outcome column

In [23]:
historical_df = df[df['outcome'].isin(['Won', 'Lost'])]

In [24]:
benchmarks = {
    "avg_sales_cycle": historical_df['sales_cycle_days'].mean(),
    "low_win_sources": historical_df.groupby('lead_source')['is_won']
                        .mean()
                        .loc[lambda x: x < x.mean()].index,
    "low_win_regions": historical_df.groupby('region')['is_won']
                        .mean()
                        .loc[lambda x: x < x.mean()].index}


the above function benchmarks can be used to identify which lead sources and regions are underperforming compared to the average win rate, as well as what the average sales cycle duration is. 
This information can help sales teams focus their efforts on improving performance in those specific areas.

In [25]:
def score_new_deals(new_df, benchmarks):
    df = new_df.copy()

    df['risk_long_cycle'] = df['sales_cycle_days'] > benchmarks['avg_sales_cycle']
    df['risk_lead_source'] = df['lead_source'].isin(benchmarks['low_win_sources'])
    df['risk_region'] = df['region'].isin(benchmarks['low_win_regions'])

    df['risk_score'] = (
        df['risk_long_cycle'].astype(int) +
        df['risk_lead_source'].astype(int) +
        df['risk_region'].astype(int)
    )

    df['risk_level'] = df['risk_score'].map({
        0: 'Low Risk',
        1: 'Medium Risk',
        2: 'Medium Risk',
        3: 'High Risk'
    })

    return df


the system uses the historical benchmarks to score new deals as they come in, providing real-time risk assessments and alerts to sales reps and managers.